In [1]:
#part one is to make functions to pull text from different types of files (txt, csv, .xlsx)

def getTextFile(textfile):
    """yield lines in the text file
    
    Args:
        textfile (path): path to the text file
    
    Yiedls: the lines in the text file with the newline character removed"""
    with open(textfile,'rb') as thefile:
        for line in thefile:
            
            yield line.strip("\n")


def getCSVFile(csvfile):
    """yield rows in the csv file
    
    Args:
        textfile (path): path to the file
    
    Yiedls: the rows in the file"""
    import csv

    with open(csvfile, 'rb') as thefile:
        csvreader = csv.reader(thefile)
        for row in csvreader:
            
            yield row


def getExcelFile(excelfile):
    """yield rows in the excel file
    
    Args:
        textfile (path): path to the file
    
    Yiedls: the rows in the file """
    import openpyxl
    
    wb = openpyxl.load_workbook(excelfile)
    sheet = wb.active
    
    for row in sheet.iter_rows():
        data = [str(cell.value) for cell in row]        
        yield data

a = getExcelFile('test.xlsx')
b = getCSVFile('A_test_csv.csv')
c = getTextFile('metadata.txt')

In [2]:
import re

def getWords(text):
    """Returns list of words in the string of text"""
    splitpattern = re.compile(r'[\s:;"<>\.!@#$%^&*\(/\\?,\)-]')
    words = splitpattern.split(text)
    for word in words:
        if word != "":
            yield word

In [3]:
import pickle
    
english_words = pickle.load(open('englishwordslist.p','rb'))
medical_words = pickle.load(open('medicalwordlist.p','rb'))
first_names = pickle.load(open('firstnameslist.p'))
last_names = pickle.load(open('lastnameslist.p'))

all_words = {key:value
             for dictionary in [english_words, medical_words]
             for key,value in dictionary.iteritems()}

all_names = {key:value
            for dictionary in [first_names, last_names]
            for key,value in dictionary.iteritems()}

In [4]:
def categorizeWord(word):
    word = word.lower()
    
    if word.isdigit():
        return 'allowed'
    allowed = is_allowed(word)
    prohibited = is_prohibited(word)
    # both false
    if allowed == True and prohibited == True or allowed == False and prohibited == False:
        return 'indeterminate'
    
    if allowed == True:
        return 'allowed'
    
    if prohibited == True:
        return 'prohibited'


def is_allowed(word):
    return word in all_words

def is_prohibited(word):
    return word in all_names

def removeDates(text):
    date_pattern = re.compile(r'[-]*[0-9]{1,2}[\W][-]*[0-9]{1,2}[\W][-]*[0-9]{2,4}')
    found_dates = date_pattern.findall(text)
    modified_text= date_pattern.sub('[REDACTED][date]', text, count=1)
    return modified_text  

In [5]:
def modifyText(text):
    words = getWords(text)
    modified_text = text[:]
    for word in words:
        category = categorizeWord(word)
        if category == 'allowed':
            pass
        if category == 'prohibited':
            modified_text = modified_text.replace(word,"[REDACTED][word]")
        if category == 'indetermnate':
            modified_text = modified_text.replace(word, "[IDETERMINATE][" + word  + "]")
    return modified_text

In [6]:
import os
import csv
import openpyxl

def makeTextFile(lines,filename):
    
    print "Writing text to path", os.getcwd() + "/" + filename
    with open(filename, 'wb') as output:
        for line in lines:
            output.write(line + "\n")
    print "Finished"
            

def makeCSVFile(rows,filename):
    print "Writing data to path", os.getcwd() + "/" + filename
    with open(filename,'wb') as output:
        csvwriter = csv.writer(output)
        for row in rows:
            csvwriter.writerow(row)
    print "Finished"

def makeExcelFile(rows, filename):
    print "Writing data to path", os.getcwd() + "/" + filename    
    wb = openpyxl.workbook.Workbook()
    sheet = wb.active
    for rownum, row in enumerate(rows):
        for colnum in range(1,len(row)+1):
            sheet.cell(row=rownum+1, column=colnum+1).value = row[colnum-1]
    wb.save(filename)
    print "Finished"

new_rows = []
outcsv = 'output.csv'
for row in b:
    one_row = []
    for text in row:
        new_text = removeDates(text)
        new_text = modifyText(new_text)
        one_row.append(new_text)
    new_rows.append(one_row)
    
makeCSVFile(new_rows, outcsv)

new_rows = []
for row in a:
    one_row = []
    for text in row:
        new_text = removeDates(text)
        new_text = modifyText(new_text)
        one_row.append(new_text)
    new_rows.append(one_row)
    
outexcel = 'output.xlsx'    
makeExcelFile(new_rows, outexcel)

new_lines = []
outtext = 'output.txt'
for line in c:
    line = removeDates(line)
    new_lines.append(modifyText(line))
makeTextFile(new_lines,outtext)

print "done"

    

Writing data to path /home/beliefs22/mynotebooks/De-Identification Program/output.csv
Finished
Writing data to path /home/beliefs22/mynotebooks/De-Identification Program/output.xlsx
Finished
Writing text to path /home/beliefs22/mynotebooks/De-Identification Program/output.txt
Finished
done
